<a href="https://colab.research.google.com/github/izzaauliyairabby/tensorflow/blob/master/DCML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup and Installation

Install TensorFlow 2.13.x: understand how to install TensorFlow and the required dependencies.
Setup Development Environment: setting up the development environment, including an IDE or Jupyter notebook.

Install TensorFlow 2
TensorFlow is tested and supported on the following 64-bit systems:

Python 3.8–3.11
Ubuntu 16.04 or later
Windows 7 or later (with C++ redistributable)
macOS 10.12.6 (Sierra) or later (no GPU support)
WSL2 via Windows 10 19044 or higher including GPUs (Experimental)

System requirements
Ubuntu 16.04 or higher (64-bit)
macOS 12.0 (Monterey) or higher (64-bit) (no GPU support)
Windows Native - Windows 7 or higher (64-bit) (no GPU support after TF 2.10)
Windows WSL2 - Windows 10 19044 or higher (64-bit)
Note: GPU support is available for Ubuntu and Windows with CUDA®-enabled cards.
Software requirements
Python 3.9–3.12
pip version 19.0 or higher for Linux (requires manylinux2014 support) and Windows. pip version 20.3 or higher for macOS.
Windows Native Requires Microsoft Visual C++ Redistributable for Visual Studio 2015, 2017 and 2019
The following NVIDIA® software are only required for GPU support.

NVIDIA® GPU drivers
>= 525.60.13 for Linux
>= 528.33 for WSL on Windows
CUDA® Toolkit 12.3.
cuDNN SDK 8.9.7.
(Optional) TensorRT to improve latency and throughput for inference.

In [ ]:
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# Or try the preview build (unstable)
!pip install tf-nightly

In [ ]:
# USING DOCKERS
#  docker pull tensorflow/tensorflow:latest  # Download latest stable image
#  docker run -it -p 8888:8888 tensorflow/tensorflow:latest-jupyter  # Start Jupyter server

# 2 Data Pipeline

- Data Loading and Preprocessing: using tf.data to load and process the dataset.
- Data Augmentation: applying data augmentation techniques to enlarge the dataset and improve the generalization ability of the model.
- Normalization and Standardization: normalize and standardize the data before using it in the model.

### 1. Data Loading and Preprocessing
Tujuan: Memuat dan memproses dataset menggunakan tf.data.

Langkah-langkah:
- Memuat Dataset
- Dataset bisa berasal dari array, file CSV, atau dataset bawaan TensorFlow.

Transformasi Dataset

- Shuffle: Mengacak data untuk menghindari bias urutan.
- Batch: Membagi data menjadi batch untuk efisiensi pelatihan.
- Prefetch: Memuat data paralel untuk mempercepat pelatihan.

In [ ]:
# DATA LOADING AND PREPROCESSING
import tensorflow as tf

# Memuat dataset MNIST
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalisasi awal (dari 0-255 ke 0-1)
train_images = train_images / 255.0
test_images = test_images / 255.0

# Membuat dataset dengan tf.data
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

### 2. Data Augmentation
Tujuan: Memperbesar dataset dengan memodifikasi data secara acak untuk meningkatkan kemampuan generalisasi model.

Teknik Umum:
- Flip: Membalik gambar secara horizontal atau vertikal.
- Rotate: Memutar gambar dengan sudut tertentu.
- Zoom: Memperbesar atau memperkecil gambar.
- Brightness: Mengubah kecerahan gambar.

In [ ]:
# DATA AUGMENTATION
import tensorflow as tf

# Fungsi augmentasi
def augment(image, label):
    """Applies random flip and brightness adjustments to the image.

    Args:
        image: The input image tensor.
        label: The corresponding label for the image.

    Returns:
        A tuple containing the augmented image and label.
    """
    image = tf.expand_dims(image, axis=-1)  # Add channel dimension if it doesn't exist
    image = tf.image.random_flip_left_right(image)  # Randomly flip the image horizontally
    image = tf.image.random_brightness(image, max_delta=0.1)  # Adjust image brightness
    return image, label

# Terapkan augmentasi pada dataset
augmented_train_dataset = train_dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

# Iterasi untuk melihat hasil augmentasi
for images, labels in augmented_train_dataset.take(1):
    print(f"Augmented image shape: {images.shape}")

### 3. Normalization and Standardization
Tujuan:

- Normalisasi: Mengubah nilai piksel menjadi rentang tertentu (misalnya 0-1).
- Standardisasi: Mengubah nilai data sehingga memiliki rata-rata 0 dan standar deviasi 1.
- Normalisasi:
Menggunakan nilai minimum dan maksimum dari dataset.

In [ ]:
# Normalisasi (0-255 ke 0-1)
train_images = train_images / 255.0
test_images = test_images / 255.0

### Standardisasi:
Menggunakan rata-rata dan standar deviasi dataset.

In [ ]:
# Standardisasi
mean = train_images.mean()
std = train_images.std()

train_images = (train_images - mean) / std
test_images = (test_images - mean) / std

### Mengintegrasikan Normalisasi dan Standardisasi dalam Pipeline:

In [ ]:
def normalize_and_standardize(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalisasi
    image = (image - tf.reduce_mean(image)) / tf.math.reduce_std(image)  # Standardisasi
    return image, label

# Terapkan pada dataset
normalized_train_dataset = train_dataset.map(normalize_and_standardize, num_parallel_calls=tf.data.AUTOTUNE)
normalized_test_dataset = test_dataset.map(normalize_and_standardize, num_parallel_calls=tf.data.AUTOTUNE)

### Pipeline Lengkap

In [ ]:
import tensorflow as tf

# Memuat dataset MNIST
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalisasi awal
train_images = train_images / 255.0
test_images = test_images / 255.0

# Membuat dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Fungsi augmentasi
def augment(image, label):
    image = tf.expand_dims(image, axis = 1)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image, label

# Fungsi normalisasi dan standardisasi
def normalize_and_standardize(image, label):
    image = tf.cast(image, tf.float32)
    image = (image - tf.reduce_mean(image)) / tf.math.reduce_std(image)
    return image, label

# Pipeline lengkap
train_dataset = (
    train_dataset
    .shuffle(10000)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .map(normalize_and_standardize, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    test_dataset
    .map(normalize_and_standardize, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

# Iterasi untuk melihat hasil akhir
for images, labels in train_dataset.take(1):
    print(f"Final image shape: {images.shape}")
    print(f"Final label shape: {labels.shape}")

# 3. TensorFlow Core Concepts

- Tensor Operations: understand basic operations on Tensors, such as - - arithmetic calculations, indexing, and reshaping.
- Gradient Descent and Backpropagation: understand the basic concepts of optimization using gradient descent and backpropagation algorithms.

### Tensors Operations

Tensor adalah struktur data utama dalam TensorFlow, mirip dengan array multidimensi.

- Operasi Dasar pada Tensor
- Arithmetic Calculations: Operasi matematika seperti penjumlahan, pengurangan, perkalian, dan pembagian.
- Indexing: Mengakses elemen tensor menggunakan indeks.
- Reshaping: Mengubah bentuk tensor tanpa mengubah data.

In [ ]:
import tensorflow as tf

# Membuat tensor
tensor_a = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
tensor_b = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

# 1. Operasi Matematika
add_result = tf.add(tensor_a, tensor_b)  # Penjumlahan
mul_result = tf.multiply(tensor_a, tensor_b)  # Perkalian elemen
matmul_result = tf.matmul(tensor_a, tensor_b)  # Perkalian matriks

print("Penjumlahan:\n", add_result)
print("Perkalian Elemen:\n", mul_result)
print("Perkalian Matriks:\n", matmul_result)

# 2. Indexing
print("Elemen pertama dari tensor_a:", tensor_a[0, 0])

# 3. Reshaping
reshaped_tensor = tf.reshape(tensor_a, [1, 4])  # Ubah bentuk menjadi 1x4
print("Tensor setelah reshaping:\n", reshaped_tensor)


### 2. Gradient Descent and Backpropagation
Gradient Descent adalah algoritma optimasi untuk meminimalkan fungsi loss.
Backpropagation adalah proses menghitung gradien untuk memperbarui bobot model.

Konsep Utama:
- Loss Function: Mengukur kesalahan antara prediksi dan target.
- Gradien: Menghitung turunan fungsi loss terhadap parameter model.
- Optimizer: Menggunakan gradien untuk memperbarui parameter model.
 Contoh Kode Gradient Descent:

In [ ]:
# Fungsi Loss (Mean Squared Error)
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Model Sederhana
w = tf.Variable(2.0)  # Inisialisasi bobot
x = tf.constant([1.0, 2.0, 3.0, 4.0])  # Input
y_true = tf.constant([2.0, 4.0, 6.0, 8.0])  # Target

# Optimizer
learning_rate = 0.01
optimizer = tf.optimizers.SGD(learning_rate)

# Gradient Descent Loop
for step in range(100):
    with tf.GradientTape() as tape:
        y_pred = w * x  # Prediksi
        loss = loss_fn(y_true, y_pred)  # Hitung loss

    # Hitung gradien
    gradients = tape.gradient(loss, [w])

    # Update parameter
    optimizer.apply_gradients(zip(gradients, [w]))

    if step % 10 == 0:
        print(f"Step {step}, Loss: {loss.numpy()}, Weight: {w.numpy()}")

# GradientTape: Merekam operasi untuk menghitung gradien.
# Optimizer: SGD digunakan untuk memperbarui bobot model.
# apply_gradients: Mengaplikasikan gradien ke parameter model.

### Contoh Backpropagation pada Neural Network

In [ ]:
# Model Sederhana
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))
])

# Compile Model
model.compile(optimizer='sgd', loss='mean_squared_error')

# Data
x_train = tf.constant([[1.0], [2.0], [3.0], [4.0]])
y_train = tf.constant([[2.0], [4.0], [6.0], [8.0]])

# Training
model.fit(x_train, y_train, epochs=200, verbose=1)

# Layer Dense: Layer linear dengan bobot yang dioptimalkan.
# SGD Optimizer: Menggunakan Stochastic Gradient Descent.
# fit: Melatih model dengan backpropagation.

# Neural Network Construction

### Sequential API
 digunakan untuk membuat model neural network sederhana dengan arsitektur berlapis secara berurutan.

In [ ]:
import tensorflow as tf

# Membuat model Sequential
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),  # Input layer
    tf.keras.layers.Dense(32, activation='relu'),  # Hidden layer
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Tampilkan arsitektur model
model.summary()

# Dense Layer: Layer fully connected.
# Activation Function: Menggunakan relu untuk hidden layer dan sigmoid untuk output.
# Compile: Mengatur optimizer, fungsi loss, dan metrik evaluasi.


### 2. Functional API
Functional API digunakan untuk membangun model yang lebih kompleks, seperti model dengan input/output ganda atau arsitektur bercabang.

In [ ]:
class CustomDenseLayer(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super(CustomDenseLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Menggunakan Custom Layer
input_layer = tf.keras.Input(shape=(10,))
custom_layer = CustomDenseLayer(64)(input_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(custom_layer)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

In [ ]:
# Contoh Custom Model:
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(32, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)

# Membuat dan melatih model
model = CustomModel()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data untuk latihan
import numpy as np
x_train = np.random.random((100, 10))
y_train = np.random.randint(2, size=(100, 1))

model.fit(x_train, y_train, epochs=10)

# Convlutional Neural Networks

- Basic CNNs: build and train CNN models for image classification.
- Pooling Layers: using pooling layers such as MaxPooling2D and AveragePooling2D to reduce data dimensionality.
- Batch Normalization and Dropout: apply batch normalization and dropout to improve model stability and performance.

### 1. Basic CNNs: Membuat dan Melatih Model CNN untuk Klasifikasi Gambar
CNN adalah jenis jaringan saraf yang digunakan untuk tugas-tugas seperti klasifikasi gambar.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Membuat model CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),  # Convolutional layer
    layers.MaxPooling2D((2, 2)),  # Max pooling
    layers.Conv2D(64, (3, 3), activation='relu'),  # Convolutional layer
    layers.MaxPooling2D((2, 2)),  # Max pooling
    layers.Flatten(),  # Flatten layer
    layers.Dense(64, activation='relu'),  # Fully connected layer
    layers.Dense(10, activation='softmax')  # Output layer (10 kelas)
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Tampilkan arsitektur model
model.summary()

# Dummy data untuk pelatihan
import numpy as np
x_train = np.random.random((100, 64, 64, 3))  # 100 gambar berukuran 64x64x3
y_train = np.random.randint(10, size=(100,))  # Label dengan 10 kelas

# Melatih model
model.fit(x_train, y_train, epochs=100, batch_size=16)


Penjelasan:
- Conv2D: Layer konvolusi untuk mengekstrak fitur dari gambar.
- MaxPooling2D: Mengurangi dimensi data dengan pooling maksimum.
- Flatten: Mengubah data 2D menjadi 1D untuk input ke fully connected layer.
- Dense: Layer fully connected untuk klasifikasi.

### 2. Pooling Layers
Pooling layer digunakan untuk mengurangi dimensi data, mempercepat komputasi, dan mencegah overfitting.

Jenis Pooling:
MaxPooling2D: Mengambil nilai maksimum dari setiap area pooling.
AveragePooling2D: Mengambil rata-rata nilai dari setiap area pooling.

Penjelasan:
- MaxPooling2D digunakan untuk mengambil fitur dominan.
- AveragePooling2D digunakan untuk mengambil rata-rata fitur.

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),  # Max pooling
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.AveragePooling2D((2, 2)),  # Average pooling
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

model.summary()

## 3. Batch Normalization and Dropout
Batch Normalization dan Dropout digunakan untuk meningkatkan stabilitas dan performa model.

Batch Normalization:
Menormalkan output dari layer sebelumnya, mempercepat pelatihan, dan mengurangi sensitivitas terhadap inisialisasi.

Dropout:
Mengabaikan beberapa neuron secara acak selama pelatihan untuk mencegah overfitting.

Contoh Kode:

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.BatchNormalization(),  # Batch normalization
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),  # Batch normalization
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout
    layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Penjelasan:
- Batch Normalization:
- Menormalkan aktivasi untuk setiap batch.
- Membantu stabilitas pelatihan.
Dropout:
- Parameter 0.5 berarti 50% neuron akan di-drop selama pelatihan.
- Mencegah overfitting dengan membuat model lebih general.

# 6. Natural Language Processing (NLP)

- Text Vectorization: using text tokenization and vectorization techniques, such as the TextVectorization layer.
- Word Embeddings: understanding and using word embeddings, such as the Embedding layer.
- Recurrent Neural Networks (RNNs): building RNN models, including LSTM and GRU, for text processing.

### 1. Text Vectorization
Text vectorization mengubah teks menjadi representasi numerik yang dapat digunakan oleh model machine learning.

TextVectorization Layer:
Layer ini digunakan untuk tokenisasi dan vektorisasi teks.

In [ ]:
from tensorflow.keras.layers import TextVectorization

# Data teks
texts = [
    "I love machine learning",
    "Natural language processing is amazing",
    "TensorFlow makes it easy"
]

# Membuat TextVectorization layer
vectorizer = TextVectorization(max_tokens=1000, output_sequence_length=10)

# Menyesuaikan layer dengan data teks
vectorizer.adapt(texts)

# Melihat hasil vektorisasi
vectorized_texts = vectorizer(texts)
print(vectorized_texts.numpy())


Penjelasan:
- max_tokens: Jumlah maksimum kata unik dalam vektor.
- output_sequence_length: Panjang maksimum setiap sequence (padding jika lebih pendek).

## 2. Word Embeddings
Word embeddings adalah representasi vektor dari kata-kata dalam ruang berdimensi rendah, menangkap hubungan semantik antar kata.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Model sederhana dengan Embedding layer
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=10),  # Embedding layer
    Flatten(),  # Mengubah vektor 3D menjadi 2D
    Dense(1, activation='sigmoid')  # Layer output
])

# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data
import numpy as np
x_train = np.random.randint(1000, size=(32, 10))  # 32 sampel, panjang sequence 10
y_train = np.random.randint(2, size=(32, 1))  # Label biner

# Melatih model
model.fit(x_train, y_train, epochs=5)


Penjelasan:
- input_dim: Ukuran vocabulary.
- output_dim: Dimensi vektor embedding.
- input_length: Panjang sequence input.

## 3. Recurrent Neural Networks (RNNs)
RNN digunakan untuk memproses data sequential, seperti teks. Dua varian populer adalah LSTM dan GRU.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Model RNN dengan LSTM
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=10),
    LSTM(128),  # LSTM layer
    Dense(1, activation='sigmoid')  # Layer output
])

# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data
x_train = np.random.randint(1000, size=(32, 10))  # 32 sampel, panjang sequence 10
y_train = np.random.randint(2, size=(32, 1))  # Label biner

# Melatih model
model.fit(x_train, y_train, epochs=5)


In [ ]:
from tensorflow.keras.layers import GRU

# Model RNN dengan GRU
model = Sequential([
    Embedding(input_dim=1000, output_dim=64, input_length=10),
    GRU(128),  # GRU layer
    Dense(1, activation='sigmoid')  # Layer output
])

# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Melatih model
model.fit(x_train, y_train, epochs=5)


Penjelasan:
- LSTM (Long Short-Term Memory):
Memiliki mekanisme gating untuk mengatasi masalah vanishing gradient.
Cocok untuk data sequence panjang.
- GRU (Gated Recurrent Unit):
Lebih ringan dibandingkan LSTM.
Menggabungkan gate input dan forget menjadi satu.

# 7. Transfer Learning

Pre-trained Models: using pre-trained models (e.g., MobileNet, InceptionV3) for image classification tasks.
Fine-tuning: fine-tuning pre-trained models to improve performance on specific datasets.
Feature Extraction: using pre-trained models as feature extractors for new classification tasks.

### 1. Pre-trained Models
Pre-trained models adalah model yang telah dilatih pada dataset besar (seperti ImageNet) dan dapat digunakan untuk tugas image classification.

Contoh dengan MobileNetV2:

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Menggunakan model pre-trained MobileNetV2 tanpa top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membuat model dengan layer tambahan
model = Sequential([
    base_model,  # Pre-trained base
    GlobalAveragePooling2D(),  # Pooling layer
    Dense(128, activation='relu'),  # Fully connected layer
    Dense(10, activation='softmax')  # Output layer untuk 10 kelas
])

# Membekukan layer pre-trained
base_model.trainable = False

# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Ringkasan model
model.summary()

### 2. Fine-tuning
Fine-tuning adalah melatih ulang sebagian dari layer pre-trained model agar dapat lebih sesuai dengan dataset spesifik.

Langkah-Langkah Fine-Tuning:
Membekukan sebagian besar layer pre-trained.
Membuka beberapa layer akhir untuk dilatih ulang pada dataset baru.
Contoh Fine-Tuning:

In [ ]:
# Membuka sebagian layer akhir untuk dilatih ulang
base_model.trainable = True

# Membekukan sebagian besar layer awal
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Kompilasi ulang model setelah membuka layer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model (gunakan dataset Anda sendiri)
# model.fit(train_dataset, validation_data=val_dataset, epochs=10)


### 3. Feature Extraction
Feature extraction menggunakan pre-trained model untuk mengekstrak fitur dari data baru tanpa melatih ulang model.

Contoh Feature Extraction:

In [ ]:
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Menggunakan MobileNetV2 untuk feature extraction
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan semua layer
base_model.trainable = False

# Memuat gambar contoh
img_path = 'example_image.jpg'  # Ganti dengan path gambar Anda
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Mengekstrak fitur
features = base_model.predict(img_array)
print("Shape of extracted features:", features.shape)


Penjelasan Utama:
Pre-trained Models:

MobileNetV2, InceptionV3, ResNet50, dll., telah dilatih pada dataset ImageNet.
Dapat digunakan langsung untuk klasifikasi atau sebagai dasar untuk tugas lain.
Fine-tuning:

Membuka sebagian layer agar dapat disesuaikan dengan dataset baru.
Berguna jika dataset baru memiliki karakteristik yang berbeda dari dataset asli.
Feature Extraction:

Menggunakan model untuk menghasilkan representasi fitur dari data baru.
Cocok untuk transfer learning dengan dataset kecil.

# 8. Time Series and Forecasting
a. Time Series Data Preparation
Persiapan data melibatkan transformasi dataset menjadi format yang dapat digunakan untuk pelatihan model.

In [ ]:
import numpy as np
import tensorflow as tf

# Fungsi untuk membuat data time series
def create_time_series(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

# Contoh dataset
data = np.sin(np.linspace(0, 100, 200))  # Sinyal sinusoidal
window_size = 10
X, y = create_time_series(data, window_size)

# Bentuk data
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


b. RNNs for Time Series
Menggunakan RNN (LSTM atau GRU) untuk memprediksi data time series.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Membuat model LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(window_size, 1)),
    Dense(1)
])

# Kompilasi model
model.compile(optimizer='adam', loss='mse')

# Latih model
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape untuk LSTM
model.fit(X, y, epochs=10, batch_size=16)


# 9. Custom Training with tf.GradientTape
a. Custom Training Loop
Menggunakan tf.GradientTape untuk melatih model secara manual.

In [ ]:
# Membuat model sederhana
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Optimizer dan loss
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.MeanSquaredError()

# Custom training loop
for epoch in range(10):
    with tf.GradientTape() as tape:
        predictions = model(X, training=True)
        loss = loss_fn(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")


### b. Custom Loss Functions
Mendefinisikan fungsi loss kustom.

In [ ]:
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))  # Mean Squared Error

# Kompilasi ulang model dengan custom loss
model.compile(optimizer='adam', loss=custom_loss)

# 10. Model Training
a. Loss Functions and Metrics
Memahami berbagai loss functions dan metrics.

In [ ]:
# Contoh loss functions
loss_fn = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.Accuracy()

# Kompilasi model
model.compile(optimizer='adam', loss=loss_fn, metrics=[metric])


### b. Early Stopping and Checkpoints
Menggunakan callback untuk menghentikan pelatihan lebih awal dan menyimpan model terbaik.

In [ ]:
# create early stopping and checkpoints

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model Checkpoint
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Dummy data for demonstration (replace with your actual data)
x_train = np.random.random((100, 10))
y_train = np.random.randint(2, size=(100, 1))

# Train the model with callbacks
model.fit(x_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping, checkpoint])

# 11. Model Evaluation and Validation
a. Confusion Matrix and Classification Report
Menggunakan sklearn untuk evaluasi.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Prediksi
y_pred = (model.predict(X) > 0.5).astype("int32")

# Confusion matrix
print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))


### ROC and AUC

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# ROC Curve
fpr, tpr, thresholds = roc_curve(y, model.predict(X))
roc_auc = auc(fpr, tpr)

# Plot
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# 12. Deployment and Serving
a. Model Saving and Loading
Menyimpan dan memuat model.

In [ ]:
# Simpan model
model.save('my_model.h5')

# Muat model
from tensorflow.keras.models import load_model
loaded_model = load_model('my_model.h5')


### b. TensorFlow Lite
Konversi model ke format TensorFlow Lite.

In [ ]:
# Konversi model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


# 13. Generative AI
a. Basic Generative AI
Contoh sederhana dengan GAN.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=100),
        layers.Dense(784, activation='sigmoid')
    ])
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=784),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()


# Finish